In [126]:
import torch
predicted_probs = torch.rand(5)
ground_truth_labels = torch.randint(0, 2, (5,))

In [127]:
x = torch.stack((1-predicted_probs, predicted_probs), dim=1) # x(是类别0的概率， 是类别1的概率)
predicted_probs, ground_truth_labels, x, x.log()


(tensor([0.2337, 0.7024, 0.3393, 0.4574, 0.5900]),
 tensor([1, 1, 0, 0, 0]),
 tensor([[0.7663, 0.2337],
         [0.2976, 0.7024],
         [0.6607, 0.3393],
         [0.5426, 0.4574],
         [0.4100, 0.5900]]),
 tensor([[-0.2662, -1.4536],
         [-1.2119, -0.3533],
         [-0.4144, -1.0810],
         [-0.6113, -0.7823],
         [-0.8917, -0.5276]]))

In [128]:
# yn * log(xn) + (1-yn) * log(1-xn)
print(nn.BCELoss(reduction='none')(predicted_probs, ground_truth_labels.float()))
-(ground_truth_labels[0] * predicted_probs[0].log() + (1-ground_truth_labels[0]) * (1-predicted_probs[0].log()))

tensor([1.4536, 0.3533, 0.4144, 0.6113, 0.8917])


tensor(1.4536)

In [129]:
# - xn
print(nn.NLLLoss(reduction='none')(x.log(), ground_truth_labels))

tensor([1.4536, 0.3533, 0.4144, 0.6113, 0.8917])


In [131]:
x = torch.randn(2, 2)
x

tensor([[ 1.2763, -0.5675],
        [ 1.9046,  1.0493]])

In [132]:
x.repeat(2, 1)

tensor([[ 1.2763, -0.5675],
        [ 1.9046,  1.0493],
        [ 1.2763, -0.5675],
        [ 1.9046,  1.0493]])

In [141]:
x

tensor([[ 1.2763, -0.5675],
        [ 1.9046,  1.0493]])

In [151]:
a = torch.tensor([[1,2,3], [4,5,6]], device="cuda")
b = torch.tensor([7.0,8.0,9.0], requires_grad=True)
a.copy_(b)
print(a)   # tensor([[7, 8, 9], [7, 8, 9]], device='cuda:0')

tensor([[7, 8, 9],
        [7, 8, 9]], device='cuda:0')


In [153]:
a

tensor([[7, 8, 9],
        [7, 8, 9]], device='cuda:0')

In [157]:
a.masked_fill_(a==7, 2)


tensor([[2, 8, 9],
        [2, 8, 9]], device='cuda:0')

In [158]:
a.grad_fn

In [159]:
b = a.clone

In [172]:
import torch
x = torch.tensor([1.0, 2.0], requires_grad=True)
y = x.clone()
y.masked_fill_(x==1.0, 3.0)  # Intermediate tensor without gradient computation
y.grad_fn# This will output None


In [5]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pretrained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Load your training data
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./bert_pretrain",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start training
trainer.train()


KeyboardInterrupt: 

In [1]:
from transformers import pipeline

2024-01-28 10:35:04.707328: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 10:35:04.946152: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 10:35:05.972321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import os
os.environ['http_proxy'] = "http://127.0.0.1:7890"
os.environ['https_proxy'] = "http://127.0.0.1:7890"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from datasets import load_dataset

In [4]:
unmasker = pipeline('fill-mask', model='bert-base-chinese')

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [11]:
unmasker("天气比较差啊！情感极性是[MASK][MASK]。")

[[{'score': 0.05212096869945526,
   'token': 2523,
   'token_str': '很',
   'sequence': '[CLS] 天 气 比 较 差 啊 ！ 情 感 极 性 是 很 [MASK] 。 [SEP]'},
  {'score': 0.04574841633439064,
   'token': 671,
   'token_str': '一',
   'sequence': '[CLS] 天 气 比 较 差 啊 ！ 情 感 极 性 是 一 [MASK] 。 [SEP]'},
  {'score': 0.042567916214466095,
   'token': 1962,
   'token_str': '好',
   'sequence': '[CLS] 天 气 比 较 差 啊 ！ 情 感 极 性 是 好 [MASK] 。 [SEP]'},
  {'score': 0.04103752598166466,
   'token': 784,
   'token_str': '什',
   'sequence': '[CLS] 天 气 比 较 差 啊 ！ 情 感 极 性 是 什 [MASK] 。 [SEP]'},
  {'score': 0.04071040451526642,
   'token': 3353,
   'token_str': '极',
   'sequence': '[CLS] 天 气 比 较 差 啊 ！ 情 感 极 性 是 极 [MASK] 。 [SEP]'}],
 [{'score': 0.2665231227874756,
   'token': 4638,
   'token_str': '的',
   'sequence': '[CLS] 天 气 比 较 差 啊 ！ 情 感 极 性 是 [MASK] 的 。 [SEP]'},
  {'score': 0.05753713101148605,
   'token': 4157,
   'token_str': '点',
   'sequence': '[CLS] 天 气 比 较 差 啊 ！ 情 感 极 性 是 [MASK] 点 。 [SEP]'},
  {'score': 0.04595167562365532,
  

In [12]:
from pprint import pprint
from paddlenlp import Taskflow
schema = [{'subject': ['object', 'value']}]
ie_en = Taskflow('information_extraction', schema=schema, model='uie-base-en')
pprint(ie_en('address to the gas_station'))

[2024-03-03 10:30:14,310] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load '/home/zhoujiaming/.paddlenlp/taskflow/information_extraction/uie-base-en'.


[{}]


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Prepare input
text = "This is a sample sample sentence for classification."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Perform inference
outputs = model(**inputs)

# Get predicted class
predicted_class = torch.argmax(outputs[0].data).item()
print(predicted_class)

2024-03-04 09:43:20.226361: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 09:43:20.262908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 09:43:20.954092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decod

0


In [17]:
model['bert'].embeddings

TypeError: 'BertForSequenceClassification' object is not subscriptable

In [2]:
outputs

(tensor([[ 0.0185, -0.0118]], grad_fn=<AddmmBackward0>),)